Final Project

PROJECT TOPIC Predict gender of baby name base on state

Authors Vincent Q. Do, Dang H. Tran, and Ryan N. Treftz


# I. Introduction:

This project is intended to make a prediction of a baby’s gender and its state.
Predicting the baby’s gender will help the local government to have a better idea
about the future workforce. This should give us some insight on how certain states
like to name their children.


# II. Data Source:

The data source we are using is ‘US Baby Names’ dataset. Only names with at
least 5 babies born in the same year/state are included in this dataset for privacy.
Which means very unique names will be excluded.

# III  Outcome and Expectations:

Create a model that will accurately predict the gender of a baby based on its
name and the state it’s borned in.
We expect the model to have somewhere around 80-90% accuracy because
there are other variables that we are disregarding such as current time period, pop
culture, and ethnicity.

Using the name, we can predict what state the babies will be born in (use state population) => Professor suggestion

# Task 1

Read in the data

In [339]:
#Proposal Solution
import pandas as pd
import math

df = pd.read_csv("StateNames.csv", usecols = ['Name', 'Year', 'Gender','State','Count'])

# Task 2

Decide on data structure to use

# Task 3

Split the data into `train, test`.
- Train names are from prior to 2000
- Test names are from 2000 to 2014

In [340]:
#Split the data into a training, devtest and test sets. (52% train, 24% devtest and 24% test)
train_names = df[(df['Year'] < 1980)]
train_names_data = pd.DataFrame(train_names.groupby(["State", "Name"])["Count"].sum()).reset_index()

devtest_names = df[(df['Year'] > 1979) & (df['Year'] < 2000)]
test_names = df[(df['Year'] >= 2000)]

# Task 4

Create the classifier:
- how many times a give name and gender appears
- calculate population of states
- prior prob

In [116]:
def population(data):
    #Get the total number of times each name appears for each state.
    state = pd.DataFrame(data.groupby(['State'])['Count'].sum()).reset_index()
    #retain the total number of records.
    country = state.Count.sum()
    return (state, country)

#Store the state dataset and country count for the training set.
state_data, country_population = population(train_names)

In [271]:
def prior_probabilities(state, total):
    return int(state_data[state_data.State == state].Count) / total

In [247]:
test_prior = prior_probabilities("PA", country_population)
test_prior

0.06594550079239292

In [61]:
state_data[state_data.State == "PA"].any().State
#state_data["State"].str.contains("PA")

True

In [385]:
def conditional_probability(data, state, name, num_of_names, vocab_size):
    #P(w|c) = count(w,c) + 1 / count(c) + |V|
    if data[data.State == state].any().State and data[data.Name == name].any().Name:
        count = int(data[(data.State == state) & (data.Name == name)].Count)
        result = (count + 1) / (num_of_names + vocab_size)
    else:
        result = 1 / (num_of_names + vocab_size)
    return result

In [289]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [378]:
def nb_classifier(name, size):
    state_prob = {}
    
    for state in states:
        state_prior = prior_probabilities(state, country_population) 
        state_cp = conditional_probability(train_names_data, state, name, country_population, size)
        state_prob[state] = math.log10(state_prior) + math.log10(state_cp) 
    
    return [key for key, value in state_prob.items() if value == max(state_prob.values())]

In [379]:
def test_classifier(data):
    total = correct = 0
    size = len(set(train_names.Name))
    for index, row in data.iterrows():
        print(row["Name"])
        guess = nb_classifier(row["Name"], size)
        print(guess, row["State"])
        if guess == row["State"]:
            correct += 1
        total += 1
    return (correct/total) * 100

In [386]:
test_classifier(devtest_names[0:5])

Jessica
['NY'] AK
Jennifer
['CA'] AK
Sarah
['NY'] AK
Amanda
['TX'] AK
Melissa
['NY'] AK


0.0

In [224]:
devtest_names[:5].Name
#test = nb_classifier(set(devtest_names.Name), state_data)
#train_names[train_names.State == "PA"].any().State and train_names[train_names.Name == "Ryan"].any().Name

0     Jessica
1    Jennifer
2       Sarah
3      Amanda
4     Melissa
Name: Name, dtype: object

In [375]:
#size = len(set(train_names.Name))
conditional_probabilities(train_names, "PA", "Aaron", country_population, size)

3.8860460805709485e-05

In [168]:
#Example of how to include gender in the returned label. Given the name predict the state and gender.
#This would require that the male and female conditional probabilites are calculated for each name in the classifier.
#example_data = pd.DataFrame(train_names.groupby(['State', 'Name', 'Gender'])['Count'].sum()).reset_index()
#state_count = int(name_data[(name_data.State == "AK") & (name_data.Name == "Aaron") & (name_data.Gender == "M")].Count)

In [370]:
#baseline
baseline = {}
for state in train_names.State:
    if state not in baseline:
        baseline[state] = 0
    baseline[state] += 1
    
max_value = [(key,value) for key, value in baseline.items() if value == max(baseline.values())]
print(max_value[0][1] / len(train_names.Name))

0.052522673656265555
